In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plb.envs import make
from diffskill.env_spec import set_render_mode

env = make('CutRearrange-v1', nn=False)
set_render_mode(env, 'CutRearrange-v1')

[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-s72hxu25
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.5
Setting pykeops dir to  /home/xingyu/.pykeops_cache/NVIDIA_GeForce_RTX_2080_Ti
[Taichi] Starting on arch=cuda
[Tina] version 0.1.1
[Tina] Taichi properties hacked
pimirives: num primitive: 2
Building primitive
action:
  dim: 3
  scale: (0.015, 0.015, 0.0)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 0.0
h: (0.15, 0.15)
init_pos: (0.5, 0.3, 0.5)
init_rot: (1.0, 0.0, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
prot: (1.0, 0.0, 0.0, 0.58)
shape: Knife
size: (0.025, 0.2, 0.06)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 0
  scale: ()
collision_group: [0.0, 0.0, 0.0]
color: (0.3, 0.3, 0.3)
friction: 0.9
h: (0.15, 0.15)
init_pos: (0.3, 0.3, 0.3)
init_rot: (1.0, 0.0, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
prot: (1.0, 0.0, 0.0, 0.58)
shape: 
upper_bound: (1.0, 1.0, 1.0)
variations: None
B

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/CutRearrange/target/target_imgs.npy'

In [ ]:
from diffskill.env_spec import get_tool_spec
tool_spec = get_tool_spec(env, 'CutRearrange-v1')

In [ ]:
import matplotlib.pyplot as plt
img = env.render('rgb', img_size=512)
print(img.shape)
plt.imshow(img[:, :, :3])
plt.show()

In [5]:
import tqdm
import glob

trajs = []

for i in range(4):
    out = sorted(glob.glob(f'sols/sol{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('sols/sol', 'data/')
        trajs.append([sol_path, goal_path, j.split('.')[0][-1]])

for i in range(4):
    out = sorted(glob.glob(f'obey/obey{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('obey/obey', 'data/')
        trajs.append([sol_path, goal_path,  j.split('.')[0][-1]])

for i in range(4):
    out = sorted(glob.glob(f'sol_remain{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('sol_remain', 'data/')
        trajs.append([sol_path, goal_path,  j.split('.')[0][-1]])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:00<00:00, 781894.22it/s]


In [6]:
tool_spec = get_tool_spec(env, 'CutRearrange-v1')

In [31]:
from sample_utils import execute
from imitation.imitation_buffer import ImitationReplayBuffer

buffer = ImitationReplayBuffer(None)

In [8]:
print(env.env.generating_cached_state)

False


In [32]:
import torch
import time
import numpy as np


for idx, (sol, goal, tid) in enumerate(tqdm.tqdm(trajs, total=len(trajs))):
    a = torch.load(sol)
    b = torch.load(goal)
    if idx == 0:
        continue

    tid = int(tid)
    action_mask = tool_spec['action_masks'][tid]
    contact_loss_mask = tool_spec['contact_loss_masks'][tid]

    initial_state = a['initial_state']
    goal_state = b[1]
    best_action = a['best_action']

    infos = []
    reset_key = {
        'init_v': idx,
        'target_v': idx,
    }

    reset_key['contact_loss_mask'] = contact_loss_mask

    state = env.reset(**reset_key) #TODO: contact loss mask..
    obs = env.render(mode='rgb')  # rgbd observation

    states, obses, actions, rewards, succs, scores = [], [], [], [], [0.], [0.]  # Append 0 for the first frame for succs and scores
    states.append(state.astype(np.float32))
    obses.append(obs)

    T = 50
    total_r = 0

    total_time = 0
    agent_time = 0
    env_time = 0
    st_time = time.time()
    actions = best_action

    _, _, _, info = env.step(np.zeros(best_action[-1].shape[-1]))
    infos = [info]


    agent_time = time.time() - st_time

    import tqdm
    for i in range(T):
        t1 = time.time()
        next_state, reward, _, info = env.step(best_action[i])
        infos.append(info)
        env_time += time.time() - t1
        states.append(next_state)
        obs = env.taichi_env.render(mode='rgb')
        obses.append(obs)
        total_r += reward
        rewards.append(reward)

        # mass_grids.append(info['mass_grid'])
    target_img = env.target_img

    emds = np.array([info['info_emd'] for info in infos])
    info_normalized_performance = np.array([info['info_normalized_performance'] for info in infos])


    total_time = time.time() - st_time
    ret = {'states': np.array(states).astype(np.float32),
           'obses': np.array(obses).astype(np.float32),
           'actions': np.array(actions).astype(np.float32),
           'target_img': target_img,
           'rewards': np.array(rewards),
           'info_rewards': np.array(rewards),
           'info_emds': emds,
           'info_final_normalized_performance': np.array(info_normalized_performance[-1]),
           'info_normalized_performance': info_normalized_performance,
           'info_total_r': total_r,
           'info_total_time': total_time,
           'info_agent_time': agent_time,
           'info_env_time': env_time,
           'action_mask': action_mask}

    ret.update(**reset_key)
    buffer.add(ret)

    if idx == 1:
        break

  0%|                                                                                                                                               | 0/2800 [00:00<?, ?it/s]

Env reseting to: /home/hza/PlasticineLab/plb/cut/cached_state_path/target/target_1.npy, init v: 1, target v: 1
emd after reset: 0.31138134002685547


  0%|                                                                                                                                    | 1/2800 [00:24<19:06:58, 24.59s/it]


In [56]:
from imitation.generate_reset_motion import generate_reset_motion
generate_reset_motion(buffer, env, reset_gripper2=True)

gen_reset_motion:   0%|                                                                                                                                | 0/1 [00:00<?, ?it/s]

Env reseting to: /home/hza/PlasticineLab/plb/cut/cached_state_path/target/target_1.npy, init v: 1, target v: 1
emd after reset: 0.31138134002685547
======================WARNING: contact loss mask not set================
reset gripper2
[0.5        0.1        0.5        0.70700002 0.         0.70700002
 0.         0.18000001] [0.6439867  0.10954438 0.19618082 0.70710677 0.         0.70710677
 0.         0.08      ]
0
reset gripper2
[0.5        0.1        0.5        0.70700002 0.         0.70700002
 0.         0.18000001] [0.6439867  0.10954438 0.19618082 0.70710677 0.         0.70710677
 0.         0.09499998]
0
reset gripper2
[0.5        0.1        0.5        0.70700002 0.         0.70700002
 0.         0.18000001] [0.6439867  0.10954438 0.19618082 0.70710677 0.         0.70710677
 0.         0.10774995]
0
reset gripper2
[0.5        0.1        0.5        0.70700002 0.         0.70700002
 0.         0.18000001] [0.6439867  0.10954438 0.19618082 0.70710677 0.         0.70710677
 0.      

gen_reset_motion: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:52<00:00, 292.49s/it]

reset gripper2
[0.5        0.1        0.5        0.70700002 0.         0.70700002
 0.         0.18000001] [0.5        0.10632683 0.49247423 0.70710677 0.         0.70710677
 0.         0.17126442]
reset primitive in 61 steps
info emds: [0.18495528 0.18550189 0.18557416 0.18556654 0.18548876 0.18549478
 0.18554288 0.18550944 0.18551563 0.18551925 0.18550979 0.18549886
 0.1854963  0.18550754 0.18551347 0.1855149  0.1855087  0.18551768
 0.18552059 0.18548852]


In [57]:
import os
buffer.save(os.path.join('dataset_path'))

dumping to  dataset_path


In [58]:
from imitation.utils import visualize_dataset
visualize_dataset('dataset_path', env.cfg.cached_state_path, os.path.join('.', 'visualization.gif'), visualize_reset=True,
                  overlay_target=True)

Loading dataset in dataset_path
Loading dataset from dataset_path
visualize_dataset, N:  1
loading img: /home/hza/PlasticineLab/plb/cut/cached_state_path/target/target_1.png
MoviePy - Building file ./visualization.gif with imageio.
